# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [6]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [7]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [10]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [11]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_project1b
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [12]:
try:
    session.set_keyspace('udacity_project1b')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1

#### Create tables partitioned by sessionId and itemInSession so that all songplays along with artists' name and song length can be displayed for each unique session and each different number of items in that session.

In [33]:
song_artist_length = """CREATE TABLE IF NOT EXISTS song_artist_length (sessionid int, 
                                                                       iteminsession int,
                                                                       artist text,
                                                                       song text,
                                                                       length float,
                                                                       PRIMARY KEY (sessionId, itemInSession))"""

try:
    session.execute(song_artist_length)
except Exception as e:
    print(e)

In [34]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_artist_length (sessionid, iteminsession, artist, song, length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [42]:
select_song_artist_length = """SELECT 
                                   artist, 
                                   song, 
                                   length, 
                                   sessionid, 
                                   iteminsession 
                             FROM 
                                   song_artist_length 
                             WHERE 
                                   sessionid = 338 and iteminsession = 4"""

try:
    rows = session.execute(select_song_artist_length)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length, row.sessionid, row.iteminsession)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875 338 4


## Query 2

#### Create tables partitioned by userId and sessionId, and each table is sorted by itemInSession. Doing so allows us to see users with number of items they included in their songplay session, broken down by each session for each user and sorted by itemInSession.

In [36]:
artist_song_user = """CREATE TABLE IF NOT EXISTS artist_song_user (userId int, 
                                                                   sessionId int, 
                                                                   itemInSession int, 
                                                                   artist text, 
                                                                   song text, 
                                                                   firstName text, 
                                                                   lastName text, 
                                                                   PRIMARY KEY ((userId, sessionId),
                                                                                 itemInSession))"""

try:
    session.execute(artist_song_user)
except Exception as e:
    print(e)

In [37]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [43]:
select_artist_song_user = """SELECT 
                                    artist,
                                    song,
                                    firstName,
                                    lastName,
                                    userId,
                                    sessionId
                             FROM 
                                    artist_song_user 
                             WHERE 
                                    userid = 10 AND sessionid = 182"""

try:
    rows = session.execute(select_artist_song_user)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname, row.userid, row.sessionid)

Down To The Bone Keep On Keepin' On Sylvie Cruz 10 182
Three Drives Greece 2000 Sylvie Cruz 10 182
Sebastien Tellier Kilometer Sylvie Cruz 10 182
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 10 182


## Query 3

#### Create tables partitioned by song and userId so that for each song we can see all the users who listened it. The reason we included userId as part of the primary key is because for any given song, there may be more than one users having the same first and last names. By adding userId ensures that each row is unique.

In [13]:
user_play_hist = """CREATE TABLE IF NOT EXISTS user_play_hist (song text, 
                                                               userId int,
                                                               firstName text,
                                                               lastName text,
                                                               PRIMARY KEY (song, userId))"""

try:
    session.execute(user_play_hist)
except Exception as e:
    print(e)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_play_hist (song, userId, firstName, lastName)"
        query = query + " VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [44]:
select_user_play_hist = """SELECT
                                   song,
                                   userId,
                                   firstName,
                                   lastName
                             FROM 
                                   user_play_hist 
                             WHERE 
                                   song = 'All Hands Against His Own'"""

try:
    rows = session.execute(select_user_play_hist)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song, row.userid, row.firstname, row.lastname)

All Hands Against His Own 29 Jacqueline Lynch
All Hands Against His Own 80 Tegan Levine
All Hands Against His Own 95 Sara Johnson


### Drop the tables before closing out the sessions

In [32]:
try:
    rows = session.execute("""DROP TABLE IF EXISTS song_artist_length""")
except Exception as e:
    print(e)
    
try:
    rows = session.execute("""DROP TABLE IF EXISTS artist_song_user""")
except Exception as e:
    print(e)  

try:
    rows = session.execute("""DROP TABLE IF EXISTS user_play_hist""")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [117]:
session.shutdown()
cluster.shutdown()